In [ ]:
from dialogue2graph.metrics.no_llm_metrics.metrics import (
    match_roles, is_correct_length
)
import json

from dialogue2graph.pipelines.model_storage import ModelStorage
from dialogue2graph.datasets.augment_dialogues.prompts import augmentation_prompt_3_vars
from dialogue2graph.pipelines.core.dialogue import DialogueMessage, Dialogue

from pydantic import BaseModel, Field
class AugmentedTurn(BaseModel):
    participant: str
    text: list[str] = Field(..., description="List of utterance variations for this turn")

class DialogueSequence(BaseModel):
    result: list[AugmentedTurn] = Field(..., description="Sequence of augmented turns")

/home/olshevskaya/miniconda3/envs/conda_env_py311/lib/python3.11/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [2]:
with open('../data/gen_dataset_augmented_0-402_nodes_edges_v5.json', "r") as fp:
    data = json.load(fp)
example = data[0]
dialogues = example['dialogues']
augmented_dialogues = example['augmented_dialogues']

dia = dialogues[0]
aug_dia = augmented_dialogues[0]

# Initialize model storage
model_storage = ModelStorage()

# Add models to storage
model_storage.add(
    key="generation-llm",
    config={"model_name": "gpt-4o-mini-2024-07-18", "temperature": 0.7},
    model_type="llm"
)

model_storage.add(
    key="formatting-llm", 
    config={"model_name": "gpt-3.5-turbo", "temperature": 0.7},
    model_type="llm"
)

INFO:dialogue2graph.pipelines.model_storage:Added llm model 'generation-llm' to storage
INFO:dialogue2graph.pipelines.model_storage:Added llm model 'formatting-llm' to storage


## augmentation_corrected.py

In [3]:
from dialogue2graph.datasets.augment_dialogues.augmentation import DialogueAugmenter

In [4]:
dialogue = Dialogue.from_list(dia['messages'])
augmenter = DialogueAugmenter(
    model_storage=model_storage,
    generation_llm="generation-llm",
    formatting_llm="formatting-llm"
)

In [7]:
result = augmenter.invoke(
    dialogue=dialogue,
    topic=example['topic'],
    prompt=augmentation_prompt_3_vars
)
result

INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"


[Dialogue(messages=[DialogueMessage(text='Hello! How can I help you today?', participant='assistant'), DialogueMessage(text='I’d like to place an order.', participant='user'), DialogueMessage(text='Awesome! Which product are you interested in?', participant='assistant'), DialogueMessage(text='I’d like to get a t-shirt.', participant='user'), DialogueMessage(text='What size do you need?', participant='assistant'), DialogueMessage(text='Medium, please.', participant='user'), DialogueMessage(text='Would you prefer that in red or blue?', participant='assistant'), DialogueMessage(text='Red, please.', participant='user'), DialogueMessage(text='Great choice! Can you provide your shipping address?', participant='assistant'), DialogueMessage(text='Sure, it’s 123 Main St.', participant='user'), DialogueMessage(text='Thanks! Your order has been confirmed. Is there anything else I can assist you with?', participant='assistant'), DialogueMessage(text='Actually, I’d like to change my order.', partic

In [5]:
result = augmenter.invoke(
    dialogue=Dialogue(messages=[]),
    topic=example['topic'],
    prompt=augmentation_prompt_3_vars
)
result

'Preprocessing failed: no messages found in the dialogue'

In [6]:
result = augmenter.invoke(
    dialogue=dialogue,
    topic=example['topic'],
    prompt=''
)
result

'Preprocessing failed: prompt should be valid instruction for LLM'

## making test data

In [52]:
with open('../data/gen_dataset_augmented_0-402_nodes_edges_v5.json', "r") as fp:
    data = json.load(fp)
dia = data[0]['dialogues'][0]
aug_result = data[0]['augmented_dialogues'][0]['messages']

In [53]:
with open('../data/gen_dataset_augmented_0-402_combined_v5.json', "r") as fp:
    data2 = json.load(fp)
augmented_dialogues = data2[0]['augmented_dialogues']
topic = dia['id']
new_aug_dias = []
for aug_dia in augmented_dialogues:
    if topic in aug_dia['id']:
        new_aug_dias.append(aug_dia)
len(new_aug_dias)

3

In [54]:
neg_aug_result = [{'participant': 'assistant',
  'text': [1, 2, 3]},
 {'participant': 'user',
  'text': ['I’d like to place an order.',
   'I want to make an order.',
   "I'm looking to order something."]},
 {'participant': 'assistant',
  'text': ['Awesome! Which product are you interested in?',
   'Sounds good! What item do you want to order?',
   'Great! What would you like to order?']},
 {'participant': 'user',
  'text': ['I would like a t-shirt.',
   "I'm interested in a t-shirt.",
   'I want to get a t-shirt.']},
 {'participant': 'assistant',
  'text': ['What size do you prefer?',
   'Which size would you like?',
   'Can you tell me your preferred size?']},
 {'participant': 'user',
  'text': ["I'd like a medium size, please.",
   'Medium would be great, thanks.',
   'Please get me a medium.']},
 {'participant': 'assistant',
  'text': ['Do you want that in red or blue?',
   'Would you prefer it in red or blue?',
   'Should I get that in red or blue for you?']},
 {'participant': 'user',
  'text': ['I prefer red, please.',
   'Red sounds good, thanks.',
   "I'll take it in red, please."]},
 {'participant': 'assistant',
  'text': ['Great choice! Could you provide your shipping address?',
   'Awesome! May I have your shipping address, please?',
   'Perfect! What’s your shipping address?']},
 {'participant': 'user',
  'text': ['Sure, it’s 123 Main Street.',
   'Of course, my address is 123 Main St.',
   'Alright, the address is 123 Main St.']},
 {'participant': 'assistant',
  'text': ['Thanks! Your order has been successfully placed. Is there anything else you need?',
   'Thank you! Your order is confirmed. Anything else I can assist you with?',
   'Thanks! Your order is on its way. Need help with anything else?']},
 {'participant': 'user',
  'text': ['Actually, I’d like to modify my order.',
   'Can I change my order, please?',
   'I was wondering if I could alter my order.']},
 {'participant': 'assistant',
  'text': ['Absolutely! What changes would you like to make?',
   'Of course! What would you like to change in your order?',
   'Sure! What do you want to modify?']},
 {'participant': 'user',
  'text': ['I want it in a large size now.',
   'I’d prefer a large size instead.',
   'Can I switch to a large size?']},
 {'participant': 'assistant',
  'text': ['What size would you like?',
   'Can you confirm the size you need?',
   'What size are you looking for now?']},
 {'participant': 'user',
  'text': ['Medium, please.',
   'I still want a medium size.',
   'Let’s stick with medium, please.']},
 {'participant': 'assistant',
  'text': ['Would you like that in red or blue?',
   'Should I put that in red or blue?',
   'Do you want it in red or blue?']},
 {'participant': 'user',
  'text': ['Red, please.', 'I’d like it in red.', 'Red works for me.']},
 {'participant': 'assistant',
  'text': ['Great! Can I have your shipping address again?',
   'Awesome! Could you provide your shipping address once more?',
   'Perfect! What’s your shipping address again?']},
 {'participant': 'user',
  'text': ['Sure, it’s 123 Main St.',
   'Of course, it’s 123 Main Street.',
   'Alright, my address is 123 Main St.']},
 {'participant': 'assistant',
  'text': ['Thank you! Your order has been successfully placed. Is there anything else I can assist you with?',
   'Thanks! Your order is confirmed. Is there anything else you need help with?',
   'Thank you! Your order is on its way. Anything else I can do for you?']},
 {'participant': 'user',
  'text': ['No, that’s everything. Thank you!',
   'That’s all for now. Thanks!',
   'Nope, that will be all. Thank you!']},
 {'participant': 'assistant',
  'text': ['Okay, feel free to reach out anytime. Have a wonderful day!',
   'Alright, don’t hesitate to contact me again. Have a great day!',
   'Sure! Reach out whenever you need. Enjoy your day!']}]

In [55]:
neg_aug_dia =  {'id': 'Responding to DMs on Instagram/Facebook._1_0_2',
  'messages': [{'participant': 'assistant',
    'text': 'Hi! Is there something I can assist you with today?'},
   {'participant': 'user', 'text': "I'm looking to order something."},
   {'participant': 'assistant',
    'text': 'Great! What would you like to order?'},
   {'participant': 'user', 'text': 'I want to get a t-shirt.'},
   {'participant': 'assistant',
    'text': 'Can you tell me your preferred size?'},
   {'participant': 'user', 'text': 'Please get me a medium.'},
   {'participant': 'assistant',
    'text': 'Thanks! Your order is on its way. Need help with anything else?'},
   {'participant': 'user',
    'text': 'I was wondering if I could alter my order.'},
   {'participant': 'assistant',
    'text': 'Perfect! What’s your shipping address again?'},
   {'participant': 'user', 'text': 'Alright, my address is 123 Main St.'},
   {'participant': 'assistant',
    'text': 'Thank you! Your order is on its way. Anything else I can do for you?'},
   {'participant': 'user', 'text': 'Nope, that will be all. Thank you!'},
   {'participant': 'assistant',
    'text': 'Sure! Reach out whenever you need. Enjoy your day!'}]}

In [56]:
D1 = Dialogue(**dia)
D2 = Dialogue(**neg_aug_dia)
print(match_roles(D1, D2))
print(is_correct_length(D1, D2))

True
False


In [59]:
test_data = []
test_data.append({'dialogue' : dia})
test_data.append({'positive_aug_result' : aug_result})
test_data.append({'negative_aug_result' : neg_aug_result})
test_data.append({'positive_aug_dialogues' : new_aug_dias})
test_data.append({'negative_aug_dialogue' : neg_aug_dia})

In [60]:
with open("../data/test_augmentation_data.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, indent=4)

In [63]:
assert 'Preprocessing failed: prompt' in 'Preprocessing failed: prompt should be a valid instruction for LLM'